In [ ]:

import h5py
import matplotlib.pyplot as plt
import numpy as np
import argparse
import importlib
import random
import os
import FLAlgorithms
import utils

#for jupyter notebook reload modules
import imp
imp.reload(FLAlgorithms) 
imp.reload(utils) 
from FLAlgorithms.servers.serveravg import FedAvg

from FLAlgorithms.servers.serverpFedMe import pFedMe
from FLAlgorithms.servers.serverperavg import PerAvg
from FLAlgorithms.trainmodel.models import *
from FLAlgorithms.servers.servertrans import pFedTrans

from utils.plot_utils import *
from utils.attention import FullQKAttention

import torch

torch.manual_seed(0)


In [ ]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

print("---------------Running time:------------",)
# Generate model
model = 'mclr'
dataset = 'Mnist'

model = Mclr_Logistic().to(device), model
algorithm = "pFedMe"
# select algorithm
batch_size = 20
learning_rate = 0.01
personal_learning_rate = 0.01
beta = 2
lamda = 15
num_glob_iters = 800
local_epochs = 20
optimizer = "SGD"
numusers = 5
K = 5
server = pFedTrans(device, dataset, algorithm, model, batch_size, learning_rate,beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, 5)
        

In [ ]:
server.emb_layer.to(device)
for user in server.users:
    user.train(10)
    user.net_values = [*user.model.state_dict().values()]
    user.per_values = user.net_values[-2:]
    value_vec = nn.utils.parameters_to_vector(user.per_values).clone()
    user.emb_vec = server.emb_layer(value_vec)

In [ ]:

import torch
import numpy as np
from kmeans_pytorch import kmeans

In [ ]:

# data
data_size, dims, num_clusters = 1000, 100, 10

client_emb_list = [user.emb_vec.data.clone().reshape(1, -1) for user in server.users]
print(len(client_emb_list))

In [ ]:

client_emb_list = torch.cat(client_emb_list, dim=0)

In [ ]:
print(client_emb_list)

In [ ]:

num_clusters = 10
# kmeans
cluster_res, cluster_centers = kmeans(
    X=client_emb_list, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
)

In [ ]:

for client_id, cluster_id in enumerate(cluster_res):
    server.users[client_id].cluster_id = cluster_res[client_id] 
    server.clusters[cluster_id].users.append(server.users[client_id])

In [ ]:

for cluster in server.clusters:
    cluster.update_model(server.emb_layer)

In [ ]:
from reformer_pytorch import *

In [ ]:
x = torch.randn((10, 1, 10)).to(device)
y = torch.randn((10, 1, 10)).to(device)
z = torch.randn((10, 1, 10)).to(device)

embed_dim = 10
num_heads = 5
attn = nn.MultiheadAttention(embed_dim, num_heads).to(device)
opt, weight = attn(x, y, z)

In [ ]:
embed_dim = 128
num_heads = 8
attn = nn.MultiheadAttention(embed_dim, num_heads).to(device)

In [21]:
#intra_cluster_attn weigh        
emb_dim = attn_dim = 128
# position embedding :weight of total samples in cluster
cluster_emb_list = [cluster.emb_vec.data.clone().reshape(1, -1) for cluster in server.clusters]
x = torch.cat(cluster_emb_list, dim=0).unsqueeze(1)
ln1 = torch.nn.LayerNorm(x.size()).to(device)
x = ln1(x)
print(x.size())
l1 = nn.Linear(emb_dim, attn_dim).to(device)
l2 = nn.Linear(emb_dim, attn_dim).to(device)
a = l1(x)
b = l2(x)
c = torch.zeros_like(a)
output, weight = attn(a, b, c)
print(weight)

torch.Size([10, 1, 128])
tensor([[[0.1003, 0.0992, 0.1001, 0.0987, 0.1003, 0.1008, 0.1002, 0.0998,
          0.1005, 0.1000],
         [0.1005, 0.0992, 0.1001, 0.0989, 0.1004, 0.1009, 0.1001, 0.0996,
          0.1005, 0.1000],
         [0.1003, 0.0991, 0.1001, 0.0989, 0.1003, 0.1010, 0.1002, 0.0997,
          0.1004, 0.1001],
         [0.1006, 0.0992, 0.0999, 0.0989, 0.1003, 0.1008, 0.1001, 0.0997,
          0.1006, 0.1000],
         [0.1004, 0.0992, 0.0998, 0.0987, 0.1005, 0.1011, 0.1003, 0.0997,
          0.1003, 0.1001],
         [0.1004, 0.0990, 0.1001, 0.0987, 0.1004, 0.1009, 0.1002, 0.0996,
          0.1005, 0.1001],
         [0.1004, 0.0988, 0.1000, 0.0989, 0.1002, 0.1010, 0.1004, 0.0997,
          0.1004, 0.1002],
         [0.1004, 0.0990, 0.1002, 0.0989, 0.1002, 0.1008, 0.1004, 0.0995,
          0.1003, 0.1003],
         [0.1005, 0.0991, 0.1000, 0.0989, 0.1003, 0.1009, 0.1002, 0.0998,
          0.1001, 0.1002],
         [0.1004, 0.0989, 0.1001, 0.0990, 0.1002, 0.1009, 0.1004, 

In [19]:
emb_dim = 128
attn_dim = 128
x = torch.randn((10, 1, 128)).to(device)
print(x.size())
l1 = nn.Linear(emb_dim, attn_dim).to(device)
l2 = nn.Linear(emb_dim, attn_dim).to(device)

a = l1(x)
b = l2(x)
c = torch.ones_like(a)
output, weight = attn(a, b, c)
print(weight)

torch.Size([10, 1, 128])
tensor([[[0.1016, 0.1019, 0.1063, 0.1005, 0.0981, 0.0958, 0.1010, 0.1004,
          0.1019, 0.0924],
         [0.0980, 0.0948, 0.0972, 0.1079, 0.0973, 0.1041, 0.0937, 0.0997,
          0.0949, 0.1124],
         [0.1008, 0.0996, 0.1008, 0.0933, 0.1097, 0.0984, 0.0969, 0.0943,
          0.1027, 0.1034],
         [0.0960, 0.0970, 0.1015, 0.0997, 0.1030, 0.1014, 0.1023, 0.1005,
          0.1079, 0.0906],
         [0.1034, 0.0964, 0.0966, 0.0999, 0.0973, 0.1046, 0.0915, 0.1065,
          0.1041, 0.0997],
         [0.1019, 0.0999, 0.1039, 0.1010, 0.1027, 0.0980, 0.0947, 0.1032,
          0.1065, 0.0882],
         [0.1024, 0.0974, 0.0950, 0.1016, 0.0994, 0.1049, 0.0952, 0.1025,
          0.1063, 0.0953],
         [0.1001, 0.0983, 0.0932, 0.1018, 0.0944, 0.1058, 0.1041, 0.1123,
          0.0955, 0.0946],
         [0.1001, 0.1044, 0.1016, 0.0929, 0.1044, 0.0966, 0.1066, 0.0942,
          0.0988, 0.1005],
         [0.1002, 0.1017, 0.0959, 0.0991, 0.0973, 0.1125, 0.0996, 

In [ ]:
inter_query = inter_query_weight(inter_attn_input)
inter_key = inter_key_weight(inter_attn_input)
inter_value = torch.ones_like(inter_query).to(device)
#inter_key = inter_key_weight(inter_attn_input)
#inter_value = inter_value_weight(inter_query)
attn_output, attn_weight = attn(inter_query, inter_key, inter_value)

In [ ]:
print(attn_weight)

In [ ]:
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
output, weight = attn(x, y, z)
print(weight)

In [ ]:
z = torch.ones((10, 1, 128)).to(device)
attn_output, attn_weight = attn(x, y, z)
print(attn_output)
print(attn_weight)

In [ ]:
torch.rand

In [ ]:
print(dot)

In [ ]:
gpu = 0
device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

for i in range(5):
    print("---------------Running time:------------",i)
    # Generate model
    model = 'mclr'
    dataset = 'Mnist'

    model = Mclr_Logistic().to(device), model
    algorithm = "pFedMe"
    # select algorithm
    batch_size = 20
    learning_rate = 0.01
    personal_learning_rate = 0.01
    beta = 2
    lamda = 15
    num_glob_iters = 800
    local_epochs = 20
    optimizer = "SGD"
    numusers = 5
    K = 5
    if(algorithm == "FedAvg"):
        server = FedAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)
        
    if(algorithm == "pFedMe"):
        server = pFedMe(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, K, personal_learning_rate, i)

    if(algorithm == "PerAvg"):
        server = PerAvg(device, dataset, algorithm, model, batch_size, learning_rate, beta, lamda, num_glob_iters, local_epochs, optimizer, numusers, i)

    server.train()
    server.test()


In [ ]:
import torch
import numpy as np
from kmeans_pytorch import kmeans

# data
data_size, dims, num_clusters = 1000, 100, 3
x = np.random.randn(data_size, dims) / 6
x = torch.from_numpy(x)

# kmeans
cluster_ids_x, cluster_centers = kmeans(
    X=x, num_clusters=num_clusters, distance='euclidean', device=torch.device('cuda:0')
)

In [ ]:

print(cluster_ids_x, cluster_centers.size())

In [ ]:
times = 5

average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

In [ ]:

user = server.users[0]
model = user.model
net_values = [*model.state_dict().values()]
print(net_values)

In [ ]:
print(server.beta)

In [ ]:

for x, y in server.users[0].testloaderfull:
    x = x.to(user.device)
    print(x.size())
    

In [ ]:

# Average data 
if(algorithm == "PerAvg"):
    algorithm == "PerAvg_p"
if(algorithm == "pFedMe"):
    average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms="pFedMe_p", batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)
average_data(num_users=numusers, loc_ep1=local_epochs, Numb_Glob_Iters=num_glob_iters, lamb=lamda,learning_rate=learning_rate, beta = beta, algorithms=algorithm, batch_size=batch_size, dataset=dataset, k = K, personal_learning_rate = personal_learning_rate,times = times)

